# Topic Categorization

This is the notebook for the tutorial ["Topic Categorization - Classic Workflow"](https://pyss3.readthedocs.io/en/latest/tutorials/topic-categorization.html#topic_classic-workflow) of [PySS3](https://pyss3.readthedocs.io).

---

Before we begin, let's import needed modules...

In [ ]:
from pyss3 import SS3
from pyss3.util import Dataset
from pyss3.server import Server

from sklearn.metrics import accuracy_score

... and unzip the "movie_review.zip" dataset inside the `datasets` folder.

In [ ]:
!unzip -u datasets/topic.zip -d datasets/

Ok, now we are ready to begin. Let's create a new SS3 instance

In [ ]:
clf = SS3()

What are the default [hyperparameter](https://pyss3.readthedocs.io/en/latest/user_guide/ss3-classifier.html#hyperparameters) values? let's see

In [ ]:
s, l, p, _ = clf.get_hyperparameters()

print("Smoothness(s):", s)
print("Significance(l):", l)
print("Sanction(p):", p)

Ok, now let's load the training and the test set using the `load_from_files` function from `pyss3.util`. Since, in this dataset, there's a single file for each category, we will use the argument ``folder_label=False`` to tell PySS3 to use each file as a different category and each line inside of it as a different document:

In [ ]:
x_train, y_train = Dataset.load_from_files("datasets/topic/train", folder_label=False)
x_test, y_test = Dataset.load_from_files("datasets/topic/test", folder_label=False)

Let's train our model...


In [ ]:
clf.fit(x_train, y_train)

Note that we don't have to create any document-term matrix! we are using just the plain `x_train` documents :D cool uh? (SS3 creates a language model for each category and therefore it doesn't need to create any document-term matrices)

Now that the model has been trained, let's test it using the documents in `x_test`

In [ ]:
y_pred = clf.predict(x_test)

Let's see how good our model performed

In [ ]:
print("Accuracy:", accuracy_score(y_pred, y_test))

Not bad using the default [hyperparameter](https://pyss3.readthedocs.io/en/latest/user_guide/ss3-classifier.html#hyperparameters) values... let's manually analyze what this model has actually learned by using the interactive "live test". Note that since we are not going to use the `x_test` for this live test**(\*)** but instead the documents in "datasets/topic/live_test", we must use the `set_testset_from_files` method to tell the server to load documents from there instead.

**(\*)** *try it if you want but since `x_test` contains (preprocessed) tweets, they don't look really good and clean.*

In [ ]:
# Server.serve(clf, x_test, y_test)  # <- this visualization doesn't look really clean and good so, instead,
                                     #    we will use the documents in "live_test" folder:

Server.set_testset_from_files("datasets/topic/live_test")

Server.serve(clf)  # <- ONLY WORKS if you're working locally, sorry :(
                   #    if you're working online through Binder, it won't work

<span style="color:red">(!) To stop the server, press `Esc` and then the `I` key twice</span>

Live test doesn't look bad, however, we will create a "more intelligent" version of this model, a version that can recognize variable-length word n-grams "on the fly". Thus, when calling the `fit` we will pass an extra argument `n_grams=3` to indicate we want SS3 to learn to recognize important words, bigrams, and 3-grams **(\*)**. Additionally, we will name our model "topic_categorization_3grams" so that we can save it and load it later from the `PySS3 Command Line` to perform the hyperparameter optimization to find better [hyperparameter](https://pyss3.readthedocs.io/en/latest/user_guide/ss3-classifier.html#hyperparameters) values.

**(\*)** *If you're curious and want to know how this is actually done by SS3, read the paper "t-SS3: a text classifier with dynamic n-grams for early risk detection over text streams" (preprint available [here](https://arxiv.org/abs/1911.06147)).*

In [ ]:
clf = SS3(name="topic_categorization_3grams")

clf.fit(x_train, y_train, n_grams=3)  # <-- note the n_grams=3 argument here

As mentioned above, we will save this trained model for later use

In [ ]:
clf.save_model()

Now let's see if the performance has improved...

In [ ]:
y_pred = clf.predict(x_test)
print("Accuracy:", accuracy_score(y_pred, y_test))

Yeah, the accuracy slightly improved but more importantly, we should now see that the model has learned "more intelligent patterns" involving sequences of words when using the interactive "live test" (like "machine learning", "artificial intelligence", "self-driving cars", etc. for the "science&technology" category). Let's see...

In [ ]:
Server.serve(clf)  # <- ONLY WORKS if you're working locally, sorry :(
                   #    if you're working online through Binder, it won't work

# (!) Remember: to stop the server, press `Esc` and then the `I` key twice

Fortunately, our model has learned to recognize these important sequences (such as "artificial intelligence" and "machine learning" in doc_2.txt,  "self-driving cars" in doc_6.txt, etc.). However, some documents aren’t perfectly classified, for instance, doc_3.txt was classified as “science&technology” (as a third topic) which is clearly wrong…

So, one last thing we are going to do is to try yo find better [hyperparameter](https://pyss3.readthedocs.io/en/latest/user_guide/ss3-classifier.html#hyperparameters) values to improve our model's performance. To achieve this, we will perform what it is known as "Hyperparameter Optimization" using the ``PySS3 Command Line`` tool.

*<span style="color:blue">At this point you should read the ["Hyperparameter Optimization"](https://pyss3.readthedocs.io/en/latest/tutorials/topic-categorization.html.html#topic_hyperparameter-optimization) section of this tutorial.</span>*

As described in the "Hyperparameter Optimization" section, we found out that the following [hyperparameter](https://pyss3.readthedocs.io/en/latest/user_guide/ss3-classifier.html#hyperparameters) values will improve our classification performance

In [ ]:
clf.set_hyperparameters(s=0.32, l=1.24, p=1.1)

Let's see if it's true...

In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
print("Accuracy:", accuracy_score(y_pred, y_test))

The accuracy has improved as expected :)

Let's perform the last check and visualize what our final model has learned and how it is classifying the documents...

In [ ]:
Server.serve(clf)  # <- ONLY WORKS if you're working locally, sorry :(
                   #    if you're working online through Binder, it won't work

# (!) Remember: to stop the server, press `Esc` and then the `I` key twice

Perfect! now the documents are classified properly! (including *doc_3.txt*) :D

...and that's it, nicely done buddy!